#ABINIT

In [ ]:
!wget https://www.abinit.org/sites/default/files/packages/abinit-8.10.3.tar.gz
!tar zxvf abinit-8.10.3.tar.gz
!mv abinit-8.10.3/ ABINIT8103
!rm abinit-8.10.3.tar.gz
%cd ABINIT8103
!./configure
!make
!make install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!zip -r /content/drive/'My Drive'/ABINIT8103.zip ./ABINIT8103

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!cp -r /content/drive/'My Drive'/ABINIT8103.zip ./
!unzip ./ABINIT8103.zip
%cd /content/ABINIT8103
!make install

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ['PATH'] = "/content/ABINIT8103/bin:"+os.environ['PATH']
!mpirun -np 2 --allow-run-as-root abinit

#QuantumESPRESSO

In [ ]:
!apt install -y gcc g++ build-essential gfortran libopenmpi-dev libblas-dev liblapack-dev libopenblas-dev libfftw3-dev libopenmpi-dev libscalapack-openmpi-dev git cmake
!wget https://github.com/QEF/q-e/archive/qe-6.4.1.tar.gz
!tar -xvzf qe-6.4.1.tar.gz
!rm qe-6.4.1.tar.gz
!mv q-e-qe-6.4.1 QuantumESPRESSO641
%cd QuantumESPRESSO641
!./configure
!make pwall

In [ ]:
import os
os.environ['PATH'] = "/content/QuantumESPRESSO641/bin:"+os.environ['PATH']
!echo $PATH
!pw.x

#DFTBplus

In [ ]:
!apt install -y libarpack2-dev libopenblas-dev
!git clone https://github.com/dftbplus/dftbplus.git
%cd dftbplus
!git submodule update --init --recursive
!./utils/get_opt_externals ALL
!mkdir _build
%cd _build
# Requires cmake 3.16+, see "Update cmake section"
!cmake -DWITH_DFTD3=true -DWITH_TRANSPORT=true -DFYPP_FLAGS="-DTRAVIS" -DWITH_ARPACK=true -DCMAKE_TOOLCHAIN_FILE=../sys/gnu.cmake ..
!make
!make install
!ctest
%cd /content/dftbplus/tools/dptools
!python3 setup.py install

In [ ]:
import os
os.environ['PATH'] = "/content/dftbplus/_build/_install/bin:"+os.environ['PATH']
os.environ['PATH'] = "/content/dftbplus/tools/misc:"+os.environ['PATH']
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
!dftb+

#Elk

In [ ]:
!wget https://downloads.sourceforge.net/project/elk/elk-7.2.42.tgz
!tar zxvf elk-7.2.42.tgz
!mv elk-7.2.42 Elk7242
!rm elk-7.2.42.tgz
%cd Elk7242/
!./setup
!sed -i 's/gfortran/mpif90/g' make.inc
!sed -i 's/-fopenmp/-fopenmp -m64 -I\/usr\/local\/lib\/include/g' make.inc
!sed -i 's/lapack.a blas.a/-lmkl_gf_lp64 -lmkl_gnu_thread -lmkl_core -lgomp -lpthread -lm -ldl/g' make.inc
!make
%cd tests
!./test-mpi.sh

In [ ]:
import os
os.environ['PATH'] = "/content/Elk7242/src:"+os.environ['PATH']
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
!mpirun -np 2 --allow-run-as-root elk < case.in

#xTB

In [ ]:
!wget https://github.com/grimme-lab/xtb/archive/refs/tags/v6.4.1.zip
!unzip v6.4.1.zip
!mv xtb-6.4.1 xTB641
%cd xTB641
import os
os.environ ['FC '] = "gfortran -O2 -mtune = native"
os.environ ['CC'] = "gcc"
!mkdir build
%cd build
!cmake -B build -DCMAKE_BUILD_TYPE=Release ../
!make
%cd ../
!make -C build install
%cd build/
!ctest

In [ ]:
import os
os.environ['XTBHOME'] = "/content/xTB"
os.environ['XTBPATH'] = os.environ['XTBHOME']+"/build:"+os.environ['XTBHOME']
os.environ['PATH'] = os.environ['XTBHOME']+"/build:"+os.environ['PATH']
os.environ['LDLP'] = os.environ['XTBHOME']+"/include:"+os.environ['XTBHOME']+"/scripts"
os.environ['LD_LIBRARY_PATH'] = os.environ['LD_LIBRARY_PATH']+":"+os.environ['LDLP']
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+":"+os.environ['XTBHOME']+"/python"
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
!xtb --help